In [ ]:
import numpy as npy
import pandas as pds
import matplotlib.pyplot as mplt
import seaborn as sbn
from fancyimpute import KNN
from sklearn.preprocessing import StandardScaler as stl
%matplotlib inline

train_data = pds.read_csv('trainData.csv')
test_data = pds.read_csv('testData.csv')
my_data = pds.concat([train_data, test_data], axis = 0)
my_data.describe()

In [ ]:
duplicate_data = my_data[my_data.duplicated()]
duplicate_data.head()

In [ ]:
print(my_data.shape)

In [ ]:
my_data.isna().sum()

In [ ]:
my_data.isna()

In [ ]:
#my_data = KNN(k=5).fit_transform(my_data)
#npy.savetxt('value.csv', my_data, delimiter=',', fmt='%0.1f', newline='\n')

In [ ]:
mplt.cla()
mplt.clf()
figure = mplt.figure(figsize = (15,20))
axes = figure.gca()
my_data.hist(ax = axes)
#mplt.savefig('histimg.png')

In [ ]:
from sklearn.feature_selection import SelectKBest as skb
from sklearn.feature_selection import chi2

attributes = my_data.iloc[:,0:15]
target_class = my_data.iloc[:,-1]

obj = skb(score_func=chi2)
values = obj.fit(attributes,target_class)
data_scores = pds.DataFrame(values.scores_)
data_columns = pds.DataFrame(attributes.columns)

attributes_scores = pds.concat([data_columns,data_scores],axis=1)
attributes_scores.columns = ['Name','Score']
attributes_scores = attributes_scores.sort_values(by='Score', ascending = False)
attributes_scores
#final_attributes = skb(chi2, k=10).fit_transform(attributes, target_class)
#print(final_attributes.shape,' ',final_attributes[0:10])

In [ ]:
mplt.cla()
mplt.clf()
mplt.figure(figsize=(5,10))
sbn.barplot(x='Score', y='Name', data=attributes_scores)
#mplt.box(False)
#mplt.grid(True)
mplt.title('Feature importance', fontsize=18)
mplt.xlabel('Importance\n ', fontsize=15)
mplt.ylabel('Features \n', fontsize=15)
mplt.rc('axes', labelsize=10)
mplt.show()

In [ ]:
mplt.cla()
mplt.clf()
corrmat = my_data.corr()
c = pds.DataFrame(corrmat.iloc[:,-1])
print(c.sort_values(by = 'TenYearCHD', ascending = False))
mplt.figure(figsize=(2,15))
sbn.heatmap(c.sort_values(by = 'TenYearCHD', ascending = False), annot = True, fmt = '.2g')
mplt.savefig('featureCorre.png')

In [ ]:
my_data2 = my_data[['sysBP', 'glucose', 'age', 'totChol', 'cigsPerDay', 'diaBP', 'prevalentHyp', 'diabetes', 'BPMeds', 'male', 'TenYearCHD']]
my_data2.head(10)

In [ ]:
mplt.cla()
mplt.clf()
sbn.pairplot(my_data2)
#mplt.savefig('pairplot.png')

In [ ]:
mplt.cla()
mplt.clf()
for y in my_data2.columns:
    sbn.boxplot(x = 'TenYearCHD', y = y, data = my_data2, linewidth = 2.5)
    #mplt.savefig('EMG {0}.png'.format(y))
    mplt.show()

In [ ]:
mplt.cla()
mplt.clf()
#my_data2[my_data2.totChol>500]
my_data2 = my_data2[my_data2.totChol<500]
sbn.boxplot(x = 'TenYearCHD', y = 'totChol', data = my_data2)
#mplt.savefig('EMG totChol2.png')

In [ ]:
mplt.cla()
mplt.clf()
#my_data2[my_data2.sysBP>290]
my_data2 = my_data2[my_data2.sysBP<290]
sbn.boxplot(x = 'TenYearCHD', y = 'sysBP', data = my_data2)
#mplt.savefig('EMG sysBP2.png')

In [ ]:
mplt.cla()
mplt.clf()
#my_data2[my_data2.cigsPerDay>65]
my_data2 = my_data2[my_data2.cigsPerDay<65]
sbn.boxplot(x = 'TenYearCHD', y = 'cigsPerDay', data = my_data2)
mplt.savefig('EMG cigsPerDay2.png')

In [ ]:
'''from sklearn.preprocessing import MinMaxScaler

scaler_obj = MinMaxScaler()
scaled_data = scaler_obj.fit_transform(my_data2)
scaled_data = pds.DataFrame(scaled_data, columns = my_data2.columns)
scaled_data.describe()'''

In [ ]:
class_value = my_data2.iloc[:,-1]
attributes = my_data2.iloc[:,0:10]

In [ ]:
'''from sklearn.model_selection import train_test_split
train_input, test_input, train_label, test_label = train_test_split(attributes, class_value, test_size=0.20, random_state = 0)'''

In [ ]:
train_input = train_data.iloc[:,0:10]
train_label = train_data.iloc[:,-1]
test_input = test_data.iloc[:,0:10]
test_label = test_data.iloc[:,-1]

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(solver = 'liblinear')
logistic_model.fit(train_input, train_label)
logistic_predicts = logistic_model.predict(test_input)

In [ ]:
accuracy = logistic_model.score(test_input, test_label)
accuracyt = logistic_model.score(train_input, train_label)
print('Logistic Regression \nTest accuracy : ', accuracy, '\nTrain accuracy : ', accuracyt)

In [ ]:
from sklearn.metrics import confusion_matrix
mplt.cla()
mplt.clf()
#mplt.figure(figsize = (8,4))
cm = confusion_matrix(test_label, logistic_predicts)
print(cm)
sbn.heatmap(cm, annot = True, fmt = 'd')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k1_model = KNeighborsClassifier(n_neighbors = 1, metric = 'chebyshev').fit(train_input, train_label)
k5_model = KNeighborsClassifier(n_neighbors = 5, metric = 'chebyshev').fit(train_input, train_label)
k10_model = KNeighborsClassifier(n_neighbors = 10, metric = 'chebyshev').fit(train_input, train_label)

k1_predicts = k1_model.predict(test_input)
k5_predicts = k5_model.predict(test_input)
k10_predicts = k10_model.predict(test_input)

In [ ]:
accu_knn1 = k1_model.score(test_input, test_label)
accu_knn5 = k5_model.score(test_input, test_label)
accu_knn10 = k10_model.score(test_input, test_label)
print('Test accuracy \nKNN 1 : ', accu_knn1, '\nKNN 5 : ', accu_knn5, '\nKNN 10 : ', accu_knn10)

In [ ]:
accut_knn1 = k1_model.score(train_input, train_label)
accut_knn5 = k5_model.score(train_input, train_label)
accut_knn10 = k10_model.score(train_input, train_label)
print('Train accuracy \nKNN 1 : ', accut_knn1, '\nKNN 5 : ', accut_knn5, '\nKNN 10 : ', accut_knn10)

In [ ]:
mplt.cla()
mplt.clf()
cm = confusion_matrix(test_label, k5_predicts)
print(cm)
sbn.heatmap(cm, annot = True, fmt = 'd')

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel = 'linear')

svm_model.fit(train_input, train_label)
svm_predicts = svm_model.predict(test_input)

In [ ]:
accu_svm = svm_model.score(test_input, test_label)
accut_svm = svm_model.score(train_input, train_label)
print('Support Vector Models \nTrain accuracy : ', accut_svm, '\nTest accuracy : ', accu_svm)

In [ ]:
mplt.cla()
mplt.clf()
cm = confusion_matrix(test_label, svm_predicts)
print(cm)
sbn.heatmap(cm, annot = True, fmt = 'd')

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(train_input, train_label)

nb_predicts = nb_model.predict(test_input)
accu_nb = nb_model.score(test_input, test_label)
accut_nb = nb_model.score(train_input, train_label)
print('Naive Bayes \nTrain accuracy : ', accut_nb, '\nTest accuracy : ', accu_nb)

In [ ]:
mplt.cla()
mplt.clf()
cm = confusion_matrix(test_label, nb_predicts)
print(cm)
sbn.heatmap(cm, annot = True, fmt = 'd')

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DTC_model = DecisionTreeClassifier(criterion = 'entropy', max_depth = 4)
DTC_model.fit(train_input, train_label)
DTC_predicts = DTC_model.predict(test_input)
accu_dtc = DTC_model.score(test_input, test_label)
accut_dtc = DTC_model.score(train_input, train_label)
print('Decision Tree\nTrain accuracy : ', accu_dtc, '\nTest accuracy : ', accut_dtc)

In [ ]:
mplt.cla()
mplt.clf()
cm = confusion_matrix(test_label, DTC_predicts)
print(cm)
sbn.heatmap(cm, annot = True, fmt = 'd')

In [ ]:
from sklearn import tree
tree.plot_tree(DTC_model.fit(train_input, train_label))